In [1]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings


c:\Users\GokulSivakumar\Desktop\Sentance_Transformation\.venv\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader


loader = CSVLoader(file_path='C:\\Users\\GokulSivakumar\\Desktop\\Sentance_Transformation\\marketing_sample_for_myntra_com-ecommerce__20190601_20190831__15k_data.csv')
documents = loader.load()

In [3]:
len(documents)

9

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [5]:
len(texts)

22

In [6]:
texts[3]

Document(page_content='complete_the_look: From House of Pataudi, this kurta is a great way to show off your style.  Dress this black piece up with your favourite accessories and head out to any family event in style.\ntype: Clothing/Men/Kurtas/House of Pataudi/More by House of Pataudi\nvariant_price: 799\nvariant_compare_at_price: 1999\nideal_for: Men\nis_in_stock: In Stock\ninventory: Shape : Straight | Sleeve Styling : Regular Sleeves | Occasion : Daily | Print or Pattern Type : Geometric | Number of Components : 1 | Design Styling : Regular | Colour Family : Monochrome | Neck : Mandarin Collar | Pattern : Printed | Body or Garment Size : Garment Measurements in | Sleeve Length : Long Sleeves | Weave Pattern : Regular | Fabric Purity : Pure | Stitch : Ready to Wear | Hemline : Straight | Wash Care : Machine Wash | Weave Type : Machine Weave | Length : Knee Length | Slit Detail : Side Slits | Fabric : Cotton\nspecifications:', metadata={'source': 'C:\\Users\\GokulSivakumar\\Desktop\\S

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

instructor_embeddings = HuggingFaceEmbeddings(
    model_name="hkunlp/instructor-xl"
)

In [8]:
from langchain.vectorstores import Chroma

In [9]:
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [10]:
vectordb=vectordb

In [11]:
vectordb.persist()
vectordb = None

In [12]:
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

In [13]:
retriever = vectordb.as_retriever()

In [14]:
docs = retriever.get_relevant_documents("unisex dominant color")

In [15]:
len(docs)

4

In [16]:
docs[0]

Document(page_content="size: XL\nbrand: The Indian Garage Co\ncare_instructions: Cotton  | Machine-wash\ndominant_material: Cotton\ntitle: The Indian Garage Co Men Blue & White Printed A-Line Kurta\nactual_color: Blue | White\ndominant_color: Blue\nproduct_type: A-Line Kurta\nbody: Blue and white printed A-line kurta, has a mandarin collar, long sleeves, zip closure, shoulder tabs, curved hem, side slits\nThe model (height 6') is wearing a size M\nCotton  Machine-wash\nFrom The Indian Garage Co, this kurta is a great way to show off your style.  You'll look dapper as ever in this blue piece paired with slim jeans and sandals, ideal for any nice family gathering.\nproduct_details: Blue and white printed A-line kurta, has a mandarin collar, long sleeves, zip closure, shoulder tabs, curved hem, side slits\nsize_fit: The model (height 6') is wearing a size M", metadata={'row': 7, 'source': 'C:\\Users\\GokulSivakumar\\Desktop\\Sentance_Transformation\\marketing_sample_for_myntra_com-ecommer

In [20]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [21]:
from dotenv import  load_dotenv
from langchain import HuggingFaceHub,LLMChain

load_dotenv()

True

In [22]:
llm=HuggingFaceHub(repo_id="google/gemma-7b-it")

In [23]:

qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [24]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [25]:
def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [26]:
# full example
query = "unisex dominant color"
llm_response = qa_chain(query)
process_llm_response(llm_response)

c:\Users\GokulSivakumar\Desktop\Sentance_Transformation\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say
that you don't know, don't try to make up an answer.

size: XL
brand: The Indian Garage Co
care_instructions: Cotton  | Machine-wash
dominant_material: Cotton
title: The Indian Garage Co Men Blue & White Printed A-Line Kurta
actual_color: Blue | White
dominant_color: Blue
product_type: A-Line Kurta
body: Blue and white printed A-line kurta, has a mandarin collar, long sleeves, zip closure, shoulder tabs,
curved hem, side slits
The model (height 6') is wearing a size M
Cotton  Machine-wash
From The Indian Garage Co, this kurta is a great way to show off your style.  You'll look dapper as ever in
this blue piece paired with slim jeans and sandals, ideal for any nice family gathering.
product_details: Blue and white printed A-line kurta, has a mandarin collar, long sleeves, zip closure,
shoulder tabs, curved hem, side slits
size_fit: The model (height 6') is wearing a size M

size